In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacityworkspace")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacityworkspace
Azure region: eastus2
Subscription id: e647143e-c9be-412b-935d-43cebc98626f
Resource group: education


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "cpu-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

try:
    cpu_cluster = ComputeTarget(ws, cluster_name)
    print("cluster exists already")
except: 
    cluster_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, cluster_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C" : choice(0.01, 0.1, 1),
    "--max_iter" : choice(20, 40, 60, 100, 150, 200)
    })

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=".", script="train.py", compute_target=cpu_cluster, environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name="Accuracy", 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs=20, 
                                     max_concurrent_runs = 4,
                                     policy=policy, run_config=src)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

hyperdrive_run.get_status()

hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_13c75d31-47e4-446e-b8ae-1db4f8a75bbe
Web View: https://ml.azure.com/runs/HD_13c75d31-47e4-446e-b8ae-1db4f8a75bbe?wsid=/subscriptions/e647143e-c9be-412b-935d-43cebc98626f/resourcegroups/education/workspaces/udacityworkspace&tid=20f82473-03b0-4b6f-a647-77b9c454d5bb

Streaming azureml-logs/hyperdrive.txt

[2022-11-12T07:40:34.303994][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-11-12T07:40:35.0392310Z][SCHEDULER][INFO]Scheduling job, id='HD_13c75d31-47e4-446e-b8ae-1db4f8a75bbe_0' 
[2022-11-12T07:40:35.1388608Z][SCHEDULER][INFO]Scheduling job, id='HD_13c75d31-47e4-446e-b8ae-1db4f8a75bbe_1' 
[2022-11-12T07:40:35.1924013Z][SCHEDULER][INFO]Scheduling job, id='HD_13c75d31-47e4-446e-b8ae-1db4f8a75bbe_2' 
[2022-11-12T07:40:35.218699][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-11-12T07:40:35.3217852Z][SCHEDULER][INFO]Scheduling job, id='HD_13c75d31-47e4-446e-b8ae-1db4f8a75bbe_3' 
[2022-11-12T

{'runId': 'HD_13c75d31-47e4-446e-b8ae-1db4f8a75bbe',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-11-12T07:40:33.846175Z',
 'endTimeUtc': '2022-11-12T08:04:36.455823Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '13d536fe-6b1b-4789-bc9a-61afcf17742d',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.44.0',
  'space_size': '18',
  'score': '0.9088012139605463',
  'best_child_run_id': 'HD_13c75d31-47e4-446e-b8ae-1db4f8a75bbe_2',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_13c75d31-47e4-446e-b8ae-1db4f8a75bbe_2'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientT

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print('Best Run Id: ', best_run.id)
print("best run metrics :",best_run.get_metrics())
print("best run details :",best_run.get_details())

Best Run Id:  HD_13c75d31-47e4-446e-b8ae-1db4f8a75bbe_2
best run metrics : {'Regularization Strength:': 0.1, 'Max iterations:': 100, 'Accuracy': 0.9088012139605463}
best run details : {'runId': 'HD_13c75d31-47e4-446e-b8ae-1db4f8a75bbe_2', 'target': 'cpu-cluster', 'status': 'Completed', 'startTimeUtc': '2022-11-12T07:56:39.905315Z', 'endTimeUtc': '2022-11-12T07:56:57.5408Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', 'ContentSnapshotId': '13d536fe-6b1b-4789-bc9a-61afcf17742d', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '0.1', '--max_iter', '100'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'cpu-cluster', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDur

In [12]:
#best_run, fitted_model = hyperdrive_run.get_output()
#joblib.dump(value=fitted_model, filename="best-trained-model.pkl")

best_fitted_model = best_run.download_file('outputs/model.pkl')

In [13]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

durl = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(durl)

In [5]:
from train import clean_data

# Use the clean_data function to clean your data.
#x, y = clean_data(### YOUR DATA OBJECT HERE ###)

#x, y = clean_data(ds)

from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
ds_cleaned = x.join(y)
ds_training_data = TabularDatasetFactory.register_pandas_dataframe(ds_cleaned, ws.get_default_datastore(), "bankmarketing-cleaned")

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/c5f70179-dcc3-49a7-83d9-3c84d22b0cad/
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'job_admin.' -> 'job_admin_'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'education_basic.4y'

In [14]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5,
    debug_log = 'automl_errors.log',
    compute_target = cpu_cluster,
    enable_early_stopping = True,
    enable_onnx_compatible_models = True)

In [15]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = exp.submit(automl_config, show_output= True)

Submitting remote run.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacityworkspace,AutoML_0c5f8323-0d8c-4726-bd87-2d07a4feade3,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

In [20]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

automl_best_run, automl_fitted_model = remote_run.get_output()
print(hasattr(automl_fitted_model, 'steps'))


Package:azureml-automl-runtime, training version:1.46.1, current version:1.44.0
Package:azureml-core, training version:1.46.0, current version:1.44.0
Package:azureml-dataprep, training version:4.5.7, current version:4.2.2
Package:azureml-dataprep-rslex, training version:2.11.4, current version:2.8.1
Package:azureml-dataset-runtime, training version:1.46.0, current version:1.44.0
Package:azureml-defaults, training version:1.46.0, current version:1.44.0
Package:azureml-interpret, training version:1.46.0, current version:1.44.0
Package:azureml-mlflow, training version:1.46.0, current version:1.44.0
Package:azureml-pipeline-core, training version:1.46.0, current version:1.44.0
Package:azureml-responsibleai, training version:1.46.0, current version:1.44.0
Package:azureml-telemetry, training version:1.46.0, current version:1.44.0
Package:azureml-train-automl-client, training version:1.46.0, current version:1.44.0
Package:azureml-train-automl-runtime, training version:1.46.1, current version:

True


In [21]:
automl_best_run.get_metrics()
automl_best_run.download_file('outputs/model.pkl', 'automl_best_model.pkl')

In [23]:
# Retrieve and save your best automl model.

from azureml.automl.runtime.onnx_convert import OnnxConverter

automl_best_run_onnx, automl_fitted_model_onnx = remote_run.get_output(return_onnx_model=True)

OnnxConverter.save_onnx_model(automl_fitted_model_onnx, './outputs/best_automl_model.onnx')

In [13]:
cpu_cluster.delete()